## CO2 Limit

In [3]:
import pypsa
import pandas as pd

In [4]:
prefix = "/mnt/c/Users/scl38887/Documents/git/aldehyde"

# Import from pypsa-earth-sec
run = "wacc15_hem_1marg_1cap_nosmr_0exp_3h_ws"
Co2Ls = ["Co2L20.0", "Co2L5.0", "Co2L2.0", "Co2L1.7", "Co2L1.2", "Co2L1.0", "Co2L0.50", "Co2L0.10"] 

carbon_stats = pd.DataFrame(columns=["constraint", "store"])

for Co2L in Co2Ls:
    n = pypsa.Network(prefix + "/workflow/subworkflows/pypsa-earth-sec/results/" + run + "/postnetworks/elec_s_10_ec_lc1.0_"+Co2L+"_3H_2030_0.15_DF_0export.nc")
    carbon_stats.loc[Co2L, "constraint"] = n.global_constraints.loc["CO2Limit", "constant"] / 1e6 # MtCO2
    carbon_stats.loc[Co2L, "store"] = n.stores_t.e.filter(like="co2 atmosphere").iloc[-1][0] / 1e6 # MtCO2


INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L20.0_3H_2030_0.15_DF_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L5.0_3H_2030_0.15_DF_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L2.0_3H_2030_0.15_DF_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L1.7_3H_2030_0.15_DF_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L1.2_3H_2030_0.15_DF_0export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_Co2L1.0_3H_2030_0.15_DF_0export.nc has buses

In [5]:
carbon_stats

,constraint,store
Co2L20.0,800.0,19.035483
Co2L5.0,200.0,19.035492
Co2L2.0,80.0,18.984209
Co2L1.7,68.0,18.956529
Co2L1.2,48.0,18.63916
Co2L1.0,40.0,17.97401
Co2L0.50,20.0,0.713903
Co2L0.10,4.0,-15.191386


### Ideas
- Instead of catching the emissions through the "store", is it possible to multiply carrier usage with their co2 intensity and sum it up?

### Investigate
- Check, wether the CO2 constraint on the carriers actually applies to the carriers by quantifying the carrier usage multiplied with their co2 intensity and summing it up
- Possible mistake: the carrier "land transport oil" is not listed in n.carriers but used for land transport -> Check with pypsa-eur!
- Why is gas/OCGT modelled as a link (as well as a generator)?

### Explaination

General
-  There is urban central solid biomass CHP CC, which captures the CO2. Does it make sense that is the same side as DAC in `balances-co2_stored` ?

PyPSA-Earth-Sec implementaion
- bla

PyPSA-Earth implementation
- The CO2 constraint is set on the carrier (on the carrier attribute "co2_emissions")
- The value in `n.stores_t.e.filter(like="co2 atmosphere").iloc[-1][0]` is equivalent to "co2" in the graph `balances-co2.pdf`. If the value is negative, it is displayed as positive in the graph
- The value in `n.stores_t.e.filter(like="co2 atmosphere").iloc[-1][0]` does not catch all emissions, since it is at roughly 20 Mt but the constraint has an effect much earlier -> `balances-co2.pdf` does not include electricity emissions (and maybe others?)
- The CO2 emission factors of the carriers are implemented correctly
- See here: [prepare_network.py](https://github.com/pypsa-meets-earth/pypsa-earth/blob/aab4d18542466e3ffd89cf5af19f5be99aabdf6a/scripts/prepare_network.py#LL74C1-L81C6): 
    ```python
    def add_co2limit(n, annual_emissions, Nyears=1.0):
        n.add(
            "GlobalConstraint",
            "CO2Limit",
            carrier_attribute="co2_emissions",
            sense="<=",
            constant=annual_emissions * Nyears,
        )
    ```

Check CHP usage

In [19]:
n.statistics().loc["Link"][n.statistics().loc["Link"].index.str.contains("CHP")].round(2)

,Capacity Factor,Capital Expenditure,Curtailment,Installed Capacity,Operational Expenditure,Optimal Capacity,Revenue,Supply,Withdrawal
carrier,,,,,,,,,
urban central gas CHP,0.21,9.638454e+04,NaN,0.0,1.96,2.23,10.27,0.19,-0.47
urban central gas CHP CC,0.70,1.492987e+09,NaN,0.0,30735.90,10449.29,139595.17,2847.34,-7300.87


Check Biogas usage

In [30]:
n.statistics().loc["Link"][n.statistics().loc["Link"].index.str.contains("bio")].round(2)

,Capacity Factor,Capital Expenditure,Curtailment,Installed Capacity,Operational Expenditure,Optimal Capacity,Revenue,Supply,Withdrawal
carrier,,,,,,,,,
biogas to gas,1.00,5.525198e+08,NaN,0.0,23607.61,7402.03,-283205.71,7401.13,-7401.13
biomass EOP,0.41,1.403531e+05,NaN,0.0,0.20,0.77,18.84,0.09,-0.31
solid biomass for industry,inf,0.000000e+00,NaN,0.0,-0.00,0.00,0.00,0.00,-0.00
solid biomass for industry CC,0.00,3.009100e+03,NaN,0.0,0.00,0.02,-0.00,0.00,-0.00
solid biomass transport,0.00,1.992530e+05,NaN,0.0,29.62,25176.53,0.00,4.63,-4.63


Show graph in Markdown
<image src="/mnt/c/Users/scl38887/Documents/git/aldehyde/workflow/subworkflows/pypsa-earth-sec/results/wacc15_hem_1marg_1cap_nosmr_0exp_3h_ws/graphs/balances-AC.pdf" />

Show graph in Markdown
<image src="/mnt/c/Users/scl38887/Documents/git/aldehyde/workflow/subworkflows/pypsa-earth-sec/results/wacc15_hem_1marg_1cap_nosmr_0exp_3h_ws/graphs/balances-DC.pdf" />

In [18]:
n.global_constraints.loc["CO2Limit"].constant

68000000.0

Check CO2 stores

In [38]:
n.stores[n.stores.index.str.contains("co2 atmosphere")]

,bus,capital_cost,carrier,e_cyclic,e_initial,e_min_pu,e_nom,e_nom_extendable,lifetime,marginal_cost,...,type,e_nom_min,e_nom_max,e_max_pu,e_initial_per_period,e_cyclic_per_period,p_set,q_set,sign,build_year
Store,,,,,,,,,,,,,,,,,,,,,
co2 atmosphere,co2 atmosphere,0.0,co2,False,0.0,-1.0,0.0,True,inf,0.00992,...,,0.0,inf,1.0,False,True,0.0,0.0,1.0,0


In [50]:
n.stores_t.e["co2 atmosphere"][-1]/1e6 # MtCO2
# equivalent to n.stores_t.e.filter(like="co2 atmosphere").iloc[-1][0]/ 1e6 # MtCO2
# equivalent to (n.stores_t.p["co2 atmosphere"]*n.snapshot_weightings.generators[0]).sum()/1e6 # MtCO2

-15.191386448535619

Check carriers (relevant for the constraint)

In [126]:
n.carriers #.index.str.contains("oil")

,co2_emissions,color,nice_name,max_growth,max_relative_growth
Carrier,,,,,
lignite,0.41,#9e5a01,Lignite,inf,0.0
biomass,0.00,#0c6013,Biomass,inf,0.0
geothermal,0.03,#ba91b1,Geothermal,inf,0.0
CCGT,0.20,#b20101,Combined-Cycle Gas,inf,0.0
solar,0.00,#f9d002,Solar,inf,0.0
onwind,0.00,#235ebc,Onshore Wind,inf,0.0
oil,0.26,#262626,Oil,inf,0.0
coal,0.34,#707070,Coal,inf,0.0
offwind-ac,0.00,#6895dd,Offshore Wind (AC),inf,0.0


Compute actual CO2 emissions (using the carriers)

Next: Include links (e.g. oil)? 
- Alternative Approach: Connect AC generators to CO2 atmosphere bus? -> modelled as links
- -> Check how pypsa-eur does it!

In [114]:
carrier = "coal"
total_usage = 0.0

# Iterate over the generators
for _, gen in n.generators.iterrows():
    total_usage += n.generators_t.p[gen.name].sum() * n.snapshot_weightings.generators[0] if gen.carrier.upper() == carrier.upper() else 0.0

# Iterate over the links
for _, link in n.links.iterrows():
    total_usage += n.links_t.p0[link.name].sum() * n.snapshot_weightings.links[0] if link.carrier.upper() == carrier.upper() else 0.0
    


print(f"Total {carrier} usage: {total_usage/1e3:.2f} GWh")


Total coal usage: 0.09 GWh


In [124]:
n.loads[n.loads.carrier == "land transport oil"]

,bus,carrier,p_set,type,q_set,sign
Load,,,,,,
MA.1.1_1_AC land transport oil,MA.1.1_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.1.2_1_AC land transport oil,MA.1.2_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.1.3_1_AC land transport oil,MA.1.3_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.10.1_1_AC land transport oil,MA.10.1_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.10.3_1_AC land transport oil,MA.10.3_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.10.4_1_AC land transport oil,MA.10.4_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.10.5_1_AC land transport oil,MA.10.5_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.11.1_1_AC land transport oil,MA.11.1_1_AC oil,land transport oil,0.0,,0.0,-1.0
MA.11.3_1_AC land transport oil,MA.11.3_1_AC oil,land transport oil,0.0,,0.0,-1.0


In [123]:
n.loads.carrier.unique()    

array(['AC', 'residential rural heat', 'services rural heat',
       'residential urban decentral heat',
       'services urban decentral heat', 'urban central heat',
       'solid biomass for industry', 'gas for industry',
       'H2 for industry', 'naphtha for industry',
       'industry oil emissions', 'low-temperature heat for industry',
       'industry electricity', 'process emissions', 'H2 for shipping',
       'shipping oil', 'shipping oil emissions', 'kerosene for aviation',
       'oil emissions', 'land transport EV', 'land transport fuel cell',
       'land transport oil', 'land transport oil emissions', 'H2'],
      dtype=object)

In [70]:
n.generators_t.p[gen.name].sum() * n.snapshot_weightings.generators[0]

18.301555418878717

In [113]:
# Links connected to the co2 atmosphere. OCGT/ Gas is added here: https://github.com/PyPSA/pypsa-eur/blob/master/scripts/prepare_sector_network.py#L767
n.links[n.links.bus2 == "co2 atmosphere"].carrier.unique() #.bus

array(['OCGT', 'biomass EOP', 'biogas to gas',
       'residential rural gas boiler', 'services rural gas boiler',
       'residential urban decentral gas boiler',
       'services urban decentral gas boiler', 'urban central gas boiler',
       'solid biomass for industry CC', 'gas for industry',
       'gas for industry CC'], dtype=object)

### Old stuff

In [ ]:
# Get the energy of all generators grouped by carrier
n.generators_t.p.groupby(n.generators.carrier,axis=1).sum().sum()/1e6*n.snapshot_weightings.generators[0] #in TWh

In [ ]:
#n.generators.p_nom_opt.groupby(n.generators.carrier).sum()/1e3

In [ ]:
# Get the energy of all loads grouped by carrier using the snapshot_weightings and loads_t.p_set
n.loads_t.p.groupby(n.loads.carrier,axis=1).sum().sum()/1e6*n.snapshot_weightings.generators[0] #in TWh

In [ ]:
# Total capacity Fischer-Tropsch    
n.links[n.links.carrier.str.contains('Fischer-Tropsch')].p_nom_opt.sum() /1e3 # in GW

In [ ]:
# Get the energy of Fischer tropsch
n.links_t.p0[n.links[n.links.carrier.str.contains('Fischer-Tropsch')].index].sum().sum()*n.snapshot_weightings.generators[0]/1e6 # in TWh

In [ ]:
n.buses_t.p

In [ ]:
n.buses_t.p[n.buses.carrier.str.contains('oil')]